# Hard Voting

In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from collections import Counter

In [23]:
## 파일 경로를 넣어주세요.
output_files = ['/data/ephemeral/32.csv', '/data/ephemeral/38.csv', '/data/ephemeral/28.csv']

## 모델의 중요도 비율을 정수로 넣어주세요.
ratio = [2, 1, 1]

In [24]:
## csv에 있는 결과들을 파일별로 ratio 만큼 반복해 output_list에 넣어줍니다. 

output_list = []

for file, ratio in zip(output_files, ratio):
    for i in range(ratio):
        output_list.append(pd.read_csv(file))

In [25]:
print('len : ', len(output_list))
print(output_list)

len :  4
[          user   item
0           11   4370
1           11   4886
2           11     47
3           11  40815
4           11  32587
...        ...    ...
313595  138493   5349
313596  138493   8970
313597  138493  32587
313598  138493   4022
313599  138493  53125

[313600 rows x 2 columns],           user   item
0           11   4370
1           11   4886
2           11     47
3           11  40815
4           11  32587
...        ...    ...
313595  138493   5349
313596  138493   8970
313597  138493  32587
313598  138493   4022
313599  138493  53125

[313600 rows x 2 columns],           user   item
0           11  40815
1           11   8961
2           11   4886
3           11   4370
4           11   7438
...        ...    ...
313595  138493   4848
313596  138493   1270
313597  138493  48394
313598  138493    593
313599  138493  33794

[313600 rows x 2 columns],           user   item
0           11  58025
1           11   2167
2           11   2174
3           11   8861
4   

In [26]:
## 유저별로 추천된 아이템들을 총망라합니다.
## len(output_list)만큼 반복하며 차곡차곡 쌓습니다.

user_recommends = dict()

for idx in tqdm(range(len(output_list))):
    grouped = output_list[idx].groupby("user")["item"]
    for user in grouped.groups.keys():
        top10 = grouped.get_group(user).tolist()
        if user in user_recommends:
            user_recommends[user].extend(top10)
        else:
            user_recommends[user] = top10

100%|██████████| 4/4 [00:06<00:00,  1.70s/it]


In [27]:
## 예시로 11번 유저의 총망라된 아이템들을 봅시다.
Counter(user_recommends[11]).most_common()

[(4370, 4),
 (4886, 4),
 (8961, 4),
 (47, 3),
 (40815, 3),
 (32587, 3),
 (7373, 3),
 (7438, 3),
 (2, 2),
 (33004, 2),
 (457, 1),
 (3703, 1),
 (58025, 1),
 (2167, 1),
 (2174, 1),
 (8861, 1),
 (53464, 1),
 (3986, 1),
 (55232, 1)]

In [28]:
## 총망라된 아이템 중 언급된 개수가 많은 순서대로 10개를 자릅니다.

final = dict()
for user in tqdm(user_recommends.keys()):
    most_common = Counter(user_recommends[user]).most_common()
    final[user] = [x[0] for x in most_common[:10]]

100%|██████████| 31360/31360 [00:00<00:00, 114930.31it/s]


In [29]:
final

{11: [4370, 4886, 8961, 47, 40815, 32587, 7373, 7438, 2, 33004],
 14: [1198, 588, 2011, 1223, 919, 1907, 1035, 4857, 1073, 4963],
 18: [1193, 296, 4235, 5995, 8873, 2324, 2692, 50, 46578, 2571],
 25: [1270, 2762, 1073, 47, 1, 7153, 1259, 2997, 1923, 2329],
 31: [6377, 79132, 68358, 52722, 8360, 91542, 68954, 31696, 34405, 2628],
 35: [5952, 2959, 44191, 48516, 4963, 4306, 33794, 3996, 2762, 6711],
 43: [17, 1197, 4973, 1196, 1210, 4963, 4896, 4886, 2858, 110],
 50: [4993, 4226, 32587, 1258, 527, 6377, 4878, 778, 750, 110],
 58: [1246, 150, 2571, 5669, 2268, 3418, 1079, 3911, 1090, 1293],
 60: [2571, 4973, 356, 58559, 1223, 6539, 4226, 30749, 50, 1198],
 61: [2959, 4979, 2918, 56367, 44195, 32587, 608, 595, 6711, 58559],
 65: [2571, 68954, 4878, 2858, 7323, 58559, 745, 6016, 32, 527],
 72: [1193, 1080, 1270, 2959, 2918, 1923, 589, 608, 1258, 1246],
 77: [4993, 2019, 1196, 6874, 1199, 1374, 589, 1080, 1372, 1136],
 82: [1089, 68157, 64839, 2028, 6377, 5956, 48780, 7361, 46578, 4886],
 85

In [30]:
## 다시 제출용 데이터프레임을 만듭니다.

submission_list = []

for user in tqdm(final.keys()):
    for item in final[user]:
        submission_list.append([user, item])


submission = pd.DataFrame(data=submission_list, columns=["user", "item"])

100%|██████████| 31360/31360 [00:00<00:00, 72962.18it/s]


In [41]:
submission.to_csv("./submission_hard.csv", index=False)